## **Machine learning Project**
### By **Thomas Marchal** #20220714, **Louis Hislaire** #  and **Jules Rougier** #20220748 


Step 1:
We choose the Dataset number D : House_data


Step 2: Scenario / About Dataset 

In this project, we chose to study a dataset about housing data. That includes numerical informations about the house (size, age, rooms, etc...), General information such as if the house is waterfront and of course the price of the house.
Using this information we can guess a few possible scenarios and choose one to study for the project.
 

<h5 style="margin-bottom: 0;"><b>Explanation:</b></h5>
<p>The target variable in this dataset is most likely price, which is a continuous numeric value. Since regression is used to predict continuous values, this points to a regression problem.</p>

<p>However, clustering could also be relevant if the goal is to group similar properties together based on features such as sqft_living, bedrooms, and bathrooms, without focusing directly on predicting price. In that case, clustering would aim to identify groups or clusters of similar houses based on these characteristics.</p>

<h5 style="margin-bottom: 0;"><b>Conclusion:</b></h5>
<p>The project could tackle a regression problem if the goal is to predict price, or a clustering problem if the goal is to group similar properties based on their features. In this project, we will be focusing on price estimation using regression techniques.</p>


Step 3: Data Loading -> Thomas


In [ ]:
# Load required librairies
import numpy as np
import pandas as pd

In [ ]:
#Load the data
df = pd.read_csv("data/House_data.csv")
df.head()

Step 4: Data Wrangling or Data Pre-processing -> Louis

Step 5: Exploratory Data Analysis -> Thomas

Step 6: Model Developement (Apply ML algorithms)
BONUS if we choose to use 2ML algorithms (see pdf) 1 normal + 2 dans les Bonus
Thomas/Louis/Jules

Step 7: Model Evaluation, Implement and compare different metrics to evaluate the quality of the model.
Chaque model Thomas/Louis/Jules

Step 8: Model Refinement -> Jules

Harmoniser le doc pour le rendu